In [1]:
import sqlite3
import os
import pandas as pd
import re
import sys
sys.path.insert(0, '../')

from xgb_process import shap_summary,xgb_model
from utils import generate_text_summary,create_context

d:\Work\Github\shap-llm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier

import dice_ml
from dice_ml.utils import helpers
import json
from pprint import pprint  
import xgboost as xgb
import shap

d:\Work\Github\shap-llm\venv\Lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [14]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
# import xgboost as xgb
# import shap
import os
import random


from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_PROJECT="DICE-LLM-Telco-Local-Explanations"


In [5]:
###Load the table and add to database
df0=pd.read_csv("..\\data\\telco_churn_data.csv",index_col=False)
df0.columns = [x.lower() for x in df0.columns]
cat_cols = ['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice',
            'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies',
            'contract', 'paperlessbilling', 'paymentmethod']
num_cols = ['tenure', 'monthlycharges', 'totalcharges']
target = 'churn'

# Ensure the target variable and all feature names in the lists are exactly as in the DataFrame
if set(cat_cols + num_cols + [target]).issubset(df0.columns):
    print("All required columns are in the DataFrame.")
else:
    missing_cols = set(cat_cols + num_cols + [target]) - set(df0.columns)
    print("Missing columns:", missing_cols)

##Ensure target is numeric
df0[target] = df0[target].apply(lambda x: 1 if x == 'Yes' else 0)
# Convert categorical columns to 'category' dtype
df0[cat_cols] = df0[cat_cols].astype('category')

# Convert numeric columns to 'numeric' dtype
for col in num_cols:
    df0[col] = pd.to_numeric(df0[col], errors='coerce')
df0['customerid']=df0['customerid'].astype('category')



# Initialize and use the model
model_instance = xgb_model.XGBoostModel(df=df0, cat_features=cat_cols, num_features=num_cols, target=target, mode='dev')
model,dtrain,X_train,dtest,X_test=model_instance.train_model()


analyzer=shap_summary.ShapAnalyzer(model=model,
                                X_train=X_train,
                                dtrain=dtrain,
                                cat_features=cat_cols,
                                num_features=num_cols)

result_df = analyzer.analyze_shap_values()
summary_df = analyzer.summarize_shap_df()

# Convert object types to category and numeric types to their respective numeric types
for col in summary_df.columns:
    if summary_df[col].dtype == 'object':
        summary_df[col] = summary_df[col].astype('category')
    elif pd.api.types.is_numeric_dtype(summary_df[col]):
        summary_df[col] = pd.to_numeric(summary_df[col])

All required columns are in the DataFrame.
ROC AUC Score: 0.84


#### Create The Dice ConterFactual Model

    - Dice doesn't support xgboost models directly. Currently it supports sklearn, pytorch and tensorflow
    - However we don't need dice to explain why the churn, but need it to tell what if
    - So we can use a base model within sklearn framework for this task
    - Then use the base model to perform the what if analysis, and select the best actions recommended by base model and implement it with xgboost mdoel to understand the relative improvmenet from top recommendations from dice model to mnimize the churn score
    - Below we will create the dice model and base model with sklearn framework
    - We will use histogram gradient booster with sklearn for base model since it takes categorical variable without encoding and is fast


In [8]:
def train_hist_model(X_train, y_train, cat_cols):
    """
    Train a HistGradientBoostingClassifier model.

    Parameters:
    X_train (DataFrame): The training data.
    y_train (Series): The target values for the training data.
    cat_cols (list): The categorical columns in the training data.

    Returns:
    HistGradientBoostingClassifier: The trained model.
    """
    # Identify categorical feature indices
    categorical_feature_indices = [X_train.columns.get_loc(col) for col in cat_cols]

    # Train a HistGradientBoostingClassifier model
    hist_model = HistGradientBoostingClassifier(categorical_features=categorical_feature_indices)
    hist_model.fit(X_train, y_train)

    return hist_model

def create_dice_model(X_train, y_train, num_cols, cat_cols, hist_model):
    """
    Create a DiCE model using a HistGradientBoostingClassifier model.

    Parameters:
    X_train (DataFrame): The training data.
    y_train (Series): The target values for the training data.
    num_cols (list): The numerical columns in the training data.
    cat_cols (list): The categorical columns in the training data.
    hist_model (HistGradientBoostingClassifier): The trained HistGradientBoostingClassifier model.

    Returns:
    tuple: A tuple containing the DiCE data object, the DiCE model object, and the DiCE explainer object.
    """
    # Create a dataframe for DiCE
    X_train_df = X_train.copy()
    X_train_df['churn'] = y_train

    # Create a DiCE data object
    d = dice_ml.Data(dataframe=X_train_df, continuous_features=num_cols, outcome_name='churn', categorical_features=cat_cols)

    # Create a DiCE model object using the HistGradientBoostingClassifier model
    m = dice_ml.Model(model=hist_model, backend='sklearn')
    exp = dice_ml.Dice(d, m)

    return d, m,exp

- Using the dice model we will create 20 what if analysis with the action context features
- Dice provides us control over : 
    1. what features to use in what if analysis, 
    2. set ranges for continous features
    3. Maximize which Class 0 /1 (or the class to maximize in multiclass)

Here's the summary of how DiCE generates counterfactual explanations using the random method, focusing on the query instance perspective:

Imagine DiCE as a "What If" Tool for Your Machine Learning Model:

- The Subject: You provide DiCE with a single example of data – your query instance. This could be information about a customer, a patient, a loan applicant, or any other data your model makes predictions on.

- Focus Areas: You tell DiCE which specific parts (features) of this data you want to explore changes in. For instance, with a customer, you might focus on their "contract type" or "monthly charges".

- Tweaking the Details: DiCE then starts to create alternative versions of your query instance by making random changes to the selected features:

    For numerical features (like income), DiCE adjusts the value up or down within a reasonable range.
    For categorical features (like payment method), DiCE switches to a different category.

- Exploring Possibilities:  Each change creates a new "what if" scenario – a slightly different version of your original data. It's like asking, "What if this customer had a different contract or paid a different amount?"

- Model's Perspective: DiCE shows each new scenario to your prediction model. This is to see how the changes might affect the model's outcome. Maybe a different contract type would lead the model to predict that a customer is less likely to leave (churn).

- Finding the Most Interesting Changes:  DiCE collects the "what if" scenarios that cause the biggest changes in your model's predictions. These are the most informative because they highlight the features that strongly influence the model's decisions.

- The "total_CFs" Parameter:

    This tells DiCE how many unique "what if" scenarios you want to see. DiCE keeps generating new scenarios until it finds that many unique ones. This helps you get a diverse range of possible outcomes.

- Other Important Parameters in explainer.generate_counterfactuals:

    features_to_vary: The list of features you want to explore changes in.
    desired_class: (Optional) If your model predicts categories, you can tell DiCE to focus on changes that lead to a specific category of prediction.
    permitted_range: (Optional) You can control the range of values DiCE can use when changing numerical features.
    Key Points:

    The random method is like a quick exploration of "what if" scenarios.
    DiCE repeats the process of tweaking features and checking predictions many times to find unique and interesting scenarios.
    This helps you understand which factors have the biggest impact on your model's predictions.
    This understanding can then be used to improve your model or make better decisions based on the model's predictions.

In [7]:
def generate_range(value):
    """
    Generate a range of values between 80% and 100% of the input value.

    Parameters:
    value (float): The input value.

    Returns:
    list: A list containing the lower and upper bounds of the range.
    """
    lower_bound = value * 0.8
    upper_bound = value * 1
    return [lower_bound, upper_bound]


def generate_dice_explanation(test_data, features_to_vary, xgb_model,explainer,total_CFs=3,desired_class=0, top_N=3, diversity_weight=1):
    """
    Generate DiCE explanations for a given test instance.

    Parameters:
    test_data (DataFrame): The test data.
    features_to_vary (list): The features to vary in the counterfactuals.
    xgb_model (XGBClassifier): The trained XGBoost model.
    explainer (DiCE): The DiCE explainer.
    total_CFs (int, optional): The total number of counterfactuals to generate. Defaults to 3.
    desired_class (int, optional): The desired class for the counterfactuals. Defaults to 0.
    top_N (int, optional): The top N counterfactuals to return. Defaults to 3.
    diversity_weight (float, optional): The weight for diversity in the counterfactuals. Defaults to 1.

    Returns:
    tuple: A tuple containing the actual instance, all counterfactuals, and the top N counterfactuals.
    """
    try:


        query_instance=test_data[xgb_model.feature_names]
        
        monthly_charge_range = generate_range(query_instance['monthlycharges'])        
        dice_exp = explainer.generate_counterfactuals(query_instance, total_CFs=total_CFs, desired_class=desired_class, features_to_vary=features_to_vary,
            permitted_range={'contract': ['One year', 'Two year'],
                             'monthlycharges': generate_range(query_instance['monthlycharges'])},
                             diversity_weight=diversity_weight,
                             random_seed=42)

        x1 = query_instance.copy()
        x1=x1[xgb_model.feature_names]
        x1['churn_prob'] = xgb_model.predict(xgb.DMatrix(x1, enable_categorical=True))
        x1['type'] = 'actual'

        x2 = dice_exp.cf_examples_list[0].final_cfs_df
        x2=x2[xgb_model.feature_names]
        x2['churn_prob'] = xgb_model.predict(xgb.DMatrix(x2, enable_categorical=True))
        x2['type'] = 'counterfactual'

        x3=x2.sort_values(by='churn_prob',ascending=True).head(top_N)
        x3=x3[x3['churn_prob']<x1['churn_prob'].values[0]]

        return x1,x2,x3
    except Exception as e:
        print(f"Could not generate counterfactual for instance {test_data.index[0]}: {e}")
        return None
    

def identify_changes_with_impact(actual, counter_factual):
    """
    Identify the changes in the counterfactuals that have an impact on the prediction.

    Parameters:
    actual (DataFrame): The actual instance.
    counter_factual (DataFrame): The counterfactual instances.

    Returns:
    str: A JSON string representing the changes and their impact.
    """
    # Initialize an empty list to store the changes
    changes = []

    # Get the actual instance
    actual_instance = actual.iloc[0]

    # Iterate over the rows of counter_factual
    for i in range(len(counter_factual)):
        # Get the counterfactual instance
        counterfactual = counter_factual.iloc[i]

        # Initialize an empty dictionary to store the changes for this row
        row_changes = {"recommendation_rank": i + 1}

        # Iterate over the features
        for feature in actual.columns:
            # If the feature value has changed and the feature is not 'type'
            if feature!='churn_prob':
                if actual_instance[feature] != counterfactual[feature] and feature != 'type':
                    # Add the change to the dictionary
                    row_changes[feature] = f'{actual_instance[feature]} -> {counterfactual[feature]}'

        # Add the impact on churn probability
        initial_prob = float(actual_instance['churn_prob'])
        new_prob = float(counterfactual['churn_prob'])
        reduction = initial_prob - new_prob
        row_changes['impact'] = f"Churn probability reduced by {reduction * 100:.2f}%"

        # Add the changes for this row to the list
        changes.append(row_changes)

    # Convert the list to a JSON object
    changes_json = json.dumps(changes, indent=4)

    return changes_json

# Visualize the counterfactuals
#dice_exp.visualize_as_dataframe(show_only_changes=True)

In [9]:
##Create the base model
hist_model = train_hist_model(X_train, dtrain.get_label(), cat_cols)
##Create the DICE explainer
_,_,exp = create_dice_model(X_train, dtrain.get_label(), num_cols, cat_cols, hist_model)

##Get the test data ready for counterfactual analysis
test_data=X_test.copy()
test_data['customerid']=test_data.index
test_data['predicted_churn_probability'] = model.predict(xgb.DMatrix(test_data[model.feature_names], enable_categorical=True))

In [12]:
##Let's test the DICE explanation for a sample query
x1,x2,x3=generate_dice_explanation(test_data=test_data.sample(1),
                          features_to_vary=['phoneservice',
       'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup',
       'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies',
       'contract', 'paperlessbilling', 'paymentmethod','monthlycharges']
       ,xgb_model=model,
       explainer=exp,
       total_CFs=20,desired_class=0,top_N=5)

pprint(identify_changes_with_impact(x1,x3))

100%|██████████| 1/1 [00:05<00:00,  5.56s/it]

('[\n'
 '    {\n'
 '        "recommendation_rank": 1,\n'
 '        "contract": "Month-to-month -> One year",\n'
 '        "paymentmethod": "Electronic check -> Bank transfer (automatic)",\n'
 '        "impact": "Churn probability reduced by 30.77%"\n'
 '    },\n'
 '    {\n'
 '        "recommendation_rank": 2,\n'
 '        "onlinesecurity": "Yes -> No",\n'
 '        "contract": "Month-to-month -> One year",\n'
 '        "impact": "Churn probability reduced by 17.26%"\n'
 '    },\n'
 '    {\n'
 '        "recommendation_rank": 3,\n'
 '        "multiplelines": "Yes -> No",\n'
 '        "monthlycharges": "79.05 -> 64.42",\n'
 '        "impact": "Churn probability reduced by 15.82%"\n'
 '    },\n'
 '    {\n'
 '        "recommendation_rank": 4,\n'
 '        "monthlycharges": "79.05 -> 67.03",\n'
 '        "impact": "Churn probability reduced by 12.96%"\n'
 '    },\n'
 '    {\n'
 '        "recommendation_rank": 5,\n'
 '        "monthlycharges": "79.05 -> 68.17",\n'
 '        "impact": "Churn p

In [13]:
##Identify the high propensity churners


pred_test=test_data.copy()
pred_test['pred']=model.predict(xgb.DMatrix(pred_test[model.feature_names], enable_categorical=True))
pred_test=pred_test[pred_test['pred']>0.9]
print(pred_test.shape)



# Create an empty DataFrame
results = pd.DataFrame(columns=['customer_id', 'changes'])

for i in range(len(pred_test)):

    test_data_instance = pred_test.iloc[[i]]
    features_to_vary = ['phoneservice',
       'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup',
       'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies',
       'contract', 'paperlessbilling', 'paymentmethod','monthlycharges']
    
   #  shap_df=shap_values[shap_values['customerid']==test_data_instance['customerid'].values[0]]
   #  print(shap_df.shape)
   #  feature_weights=get_feature_weights_from_df(shap_df=shap_df,row_index=0, selected_features=features_to_vary)
   #  print(feature_weights)
    
    actual, counterfactual, top_counterfactuals = generate_dice_explanation(test_data=test_data_instance, 
                                                                            features_to_vary=features_to_vary, 
                                                                            xgb_model=model,
                                                                            explainer=exp, 
                                                                            total_CFs=20, 
                                                                            desired_class=0,
                                                                            top_N=5, 
                                                                            diversity_weight=1)
    changes = identify_changes_with_impact(actual, top_counterfactuals)
    print(f"Changes for instance {i}:\n")
    # Create a DataFrame for the results and concatenate it with results
    results = pd.concat([results, pd.DataFrame({'customer_id': [test_data_instance['customerid'].values[0]], 'changes': [changes]})], ignore_index=True)

(47, 22)


100%|██████████| 1/1 [00:01<00:00,  1.70s/it]


Changes for instance 0:



100%|██████████| 1/1 [00:00<00:00,  2.78it/s]


Changes for instance 1:



...

Changes for instance 44:



100%|██████████| 1/1 [00:00<00:00,  2.47it/s]


Changes for instance 45:



100%|██████████| 1/1 [00:00<00:00,  2.97it/s]

Changes for instance 46:



#### LLM to explain the Recommendation

Dice Explanation is working great. We have the top 5 best actions to be made to minimize churn by x% as detailed by our xgboost model. 
Now, we can explain why the churn happens from SHAP and what to do to prevent churn from DICE. Let's pass this info to the LLM agent and use it to create Notes for Agent

In [15]:
##Let's prepare the data, SHAP data and Dice Data to use with GPT Model
data=pred_test.copy()
data=data.rename(columns={'pred':'predicted_churn'})
dice_data=results.copy()


test_analyzer=shap_summary.ShapAnalyzer(model=model,
                                X_train=data[model.feature_names],
                                dtrain=xgb.DMatrix(data[model.feature_names], enable_categorical=True),
                                cat_features=cat_cols,
                                num_features=num_cols)

shap_data=pd.DataFrame(test_analyzer.get_shap_value(),columns=model.feature_names)


shap_data['customerid']=data['customerid'].values


In [16]:
##Let's look at DICE reccomendation for one of the customers
pprint(dice_data['changes'].values[1])
#Looks great. It correctly  reccomends best action is to Move Contract from Month-To_Mont to Two Years and monthly charges from 85 to 72, which will reduce the churn probability by 90.72%

('[\n'
 '    {\n'
 '        "recommendation_rank": 1,\n'
 '        "contract": "Month-to-month -> Two year",\n'
 '        "monthlycharges": "85.55 -> 72.65",\n'
 '        "impact": "Churn probability reduced by 92.93%"\n'
 '    },\n'
 '    {\n'
 '        "recommendation_rank": 2,\n'
 '        "contract": "Month-to-month -> Two year",\n'
 '        "paymentmethod": "Electronic check -> Credit card (automatic)",\n'
 '        "impact": "Churn probability reduced by 90.72%"\n'
 '    },\n'
 '    {\n'
 '        "recommendation_rank": 3,\n'
 '        "onlinesecurity": "No -> No internet service",\n'
 '        "contract": "Month-to-month -> Two year",\n'
 '        "impact": "Churn probability reduced by 89.70%"\n'
 '    },\n'
 '    {\n'
 '        "recommendation_rank": 4,\n'
 '        "contract": "Month-to-month -> Two year",\n'
 '        "monthlycharges": "85.55 -> 74.1",\n'
 '        "impact": "Churn probability reduced by 88.85%"\n'
 '    },\n'
 '    {\n'
 '        "recommendation_rank": 5,\

In [17]:
##Let's design the prompt and GPT model chain
def create_explanation_prompt(data,shap_data,dice_data,n=None):
    """
    Create a prompt for the GPT model to generate an explanation.

    Parameters:
    data (DataFrame): The customer data.
    shap_data (DataFrame): The SHAP values for each feature.
    dice_data (DataFrame): The counterfactuals generated by DiCE.
    n (int, optional): The index of the customer to generate an explanation for. If not provided, a random index is chosen.

    Returns:
    str: The generated prompt.
    """

    if n==None:
        n = random.randint(0, dice_data.shape[0] - 1)

    predicted_churn=data['predicted_churn'].values[n]
    customer = data.drop(columns={'predicted_churn'},axis=0).iloc[n]
    shap = shap_data.iloc[n]
    dice = dice_data.iloc[n]

    # Create a formatted string where each column name is followed by its value
    customer_string = ', '.join([f"{col}={value}" for col, value in customer.items()])
    shap_string = ', '.join([f"{col}={value}" for col, value in shap.items()])
    dicestring = dice_data['changes'].values[n]

    prompt = f"""
            YOU ARE AN ASSISTANT TO A CUSTOMER SERVICE AGENT.

            Here are the details for a customer the agent is assisting:
            - Customer Features: {customer_string}
            - Model's Predicted Churn Probability: {predicted_churn}
            - Individual SHAP Contributions for Each Feature: {shap_string}
            - Top Reccomended actions to Reduce Churn from CounterFactual Analysis: {dicestring}


            
            Here is the data dicntionary for the dataset:
            Customer ID: Unique customer identifier
            gender: Whether the customer is a male or a female
            SeniorCitizen: Whether the customer is a senior citizen or not (1, 0)
            Partner: Whether the customer has a partner or not (Yes, No)
            Dependents: Whether the customer has dependents or not (Yes, No)
            tenure: Number of months the customer has stayed with the company
            PhoneService: Whether the customer has a phone service or not (Yes, No)
            MultipleLines: Whether the customer has multiple lines or not (Yes, No, No phone service)
            InternetService: Customer’s internet service provider (DSL, Fiber optic, No)
            OnlineSecurity: Whether the customer has online security or not (Yes, No, No internet service)
            OnlineBackup: Whether the customer has online backup or not (Yes, No, No internet service)
            DeviceProtection: Whether the customer has device protection or not (Yes, No, No internet service)
            TechSupport: Whether the customer has tech support or not (Yes, No, No internet service)
            StreamingTV: Whether the customer has streaming TV or not (Yes, No, No internet service)
            StreamingMovies: Whether the customer has streaming movies or not (Yes, No, No internet service)
            Contract: The contract term of the customer (Month-to-month, One year, Two year)
            PaperlessBilling: Whether the customer has paperless billing or not (Yes, No)
            PaymentMethod: The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
            MonthlyCharges: The amount charged to the customer monthly
            TotalCharges: The total amount charged to the customer
            Churn: Whether the customer churned or not (Yes or No)
            predicted_churn: the predicted churn probability for the customer by the model


            Based on this information, explain to the agent in non-technical terms:
            1. Provide summary of who the customer is.
                - A short concise summary in under 50 words.
            2. What is the customer's predicted churn probability?
            Answer in 10 words.
            3. Identify the top 3 reasons for the customer's potential churn from the individual shap contribution. Provide a brief explanation (within 50 words for each) of why these 
                features significantly influence the churn prediction.

            4. Suggest the top 5 actions the agent can take to reduce the likelihood of churn, based on the counterfactual analysis. Each suggestion should include:
            - Provide the whole counterfactual recommendation for each action.
            - If recommendation is to change contraact to two year and Change device protection to "No internet service" mention both of them
            - DONOT omit information from the counterfactuals
            - A concise recommendation (50 words)
            - Provide an estimated churn reduction made by this action. Make this estimate from the shap value for each feature.
            - Justify why and how this action helps reduce churn by the % you have mentioned, based solely on the data provided and stats.
            - Explain this in simple words for agent to understand with a justification of the churn reduction.


            Remember:
            - The magnitude of a SHAP value indicates the strength of a feature's influence on the prediction.
            - **Positive SHAP values increase the likelihood of churn; negative values decrease it.**
            - **Positive SHAP values increase the likelihood of churn; negative values decrease it.**
            - Recommendations should be strictly based on the information provided in the SHAP contributions , customer features and patterns from decision tree stats.
            """

    
    return prompt


parser = StrOutputParser()

chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
chain = (
    chat_model
    | StrOutputParser()
)

def agent_response(N=49):

    """
    Generate an agent response using the GPT model.

    Parameters:
    N (int, optional): The index of the customer to generate a response for. Defaults to 49.

    Returns:
    None
    """

    response=chain.invoke(create_explanation_prompt(data=data,shap_data=shap_data,dice_data=dice_data,n=N))
    parts = response.split('\n\n')

    # Extract each part
    who_is_customer = parts[0].strip()
    churn_probability_explanation = parts[1].strip()
    reasons_for_churn = ' '.join(parts[2:3]).strip()
    action_recommendations = ' '.join(parts[3:4]).strip()

    print(f"""Who is the customer?\n{who_is_customer}\n\n""")
    print(f"""Likelihood for Churn\n{churn_probability_explanation}\n\n""")
    print(f"""Probable Reasons for Churn\n{reasons_for_churn}\n\n""")
    print(f"""Recommended Actions to Prevent Churn\n{action_recommendations}\n\n""")

Let's Run the Chain and get the results for a couple of customers

In [18]:
agent_response(N=20)

Who is the customer?
1. Summary of the customer: This male customer is a new subscriber with fiber optic internet, streaming TV, high monthly charges, and a high predicted churn probability.


Likelihood for Churn
2. Predicted churn probability: High at 91.97%.


Probable Reasons for Churn
3. Top 3 reasons for potential churn:
   - Month-to-month contract: Customers on short contracts are more likely to churn due to the lack of commitment.
   - No online security: Lack of online security can lead to dissatisfaction and potential security issues.
   - No internet service for streaming movies: Missing out on a popular service may lead to dissatisfaction and churn.


Recommended Actions to Prevent Churn
4. Top 5 actions to reduce churn:
   1. Change online backup to "No internet service" and contract to "Two year."
      - Churn reduction by 90.23%.
      - By offering a long-term contract and removing unnecessary services, the customer is more likely to stay committed.
   2. Change tech 

In [19]:
agent_response(N=7)

Who is the customer?
1. Summary of the customer:
   The customer is a male with no partner or dependents, using DSL internet service. He has no phone service, online security, online backup, tech support, and device protection. He watches streaming movies, has a month-to-month contract, and prefers paperless billing.


Likelihood for Churn
2. Predicted churn probability:
   High churn probability at 96.5%.


Probable Reasons for Churn
3. Top 3 reasons for potential churn:
   - Long contract term (Month-to-month) increases churn as customers may prefer flexibility.
   - Lack of tech support impacts churn negatively, as customers value assistance.
   - Streaming movies positively affect churn; customers may consider this a valuable service.


Recommended Actions to Prevent Churn
4. Top 5 actions to reduce churn:
   1. Change tech support to "No internet service" & contract to "Two year".
      - Churn reduction by 93.73%.
      - By offering a longer contract with no tech support, custom